In [3]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D
import cv2

# Load and preprocess the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1).astype('float32')
X_train /= 255
X_test /= 255

number_of_classes = 10
y_train = to_categorical(y_train, number_of_classes)
y_test = to_categorical(y_test, number_of_classes)

# Create the model
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(X_train.shape[1], X_train.shape[2], 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(number_of_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=200)

# Load and preprocess the test image
img = cv2.imread('5.jpeg', cv2.IMREAD_GRAYSCALE)
resized = cv2.resize(img, (28, 28), interpolation=cv2.INTER_AREA)
img = np.resize(resized, (28, 28, 1))
im2arr = np.array(img)
im2arr = im2arr.reshape(1, 28, 28, 1)

# Make predictions and get the digit output
y_pred = model.predict(im2arr)
digit = np.argmax(y_pred)

print("Predicted Digit:", digit)

Epoch 1/5
300/300 [==============================] - 14s 43ms/step - loss: 0.4870 - accuracy: 0.8422 - val_loss: 0.0813 - val_accuracy: 0.9756
Epoch 2/5
300/300 [==============================] - 13s 42ms/step - loss: 0.1479 - accuracy: 0.9548 - val_loss: 0.0547 - val_accuracy: 0.9816
Epoch 3/5
300/300 [==============================] - 13s 42ms/step - loss: 0.1160 - accuracy: 0.9652 - val_loss: 0.0427 - val_accuracy: 0.9871
Epoch 4/5
300/300 [==============================] - 13s 42ms/step - loss: 0.0980 - accuracy: 0.9695 - val_loss: 0.0364 - val_accuracy: 0.9873
Epoch 5/5
1/1 [==============================] - 0s 81ms/step
Predicted Digit: 5


In [1]:
import random
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import gym

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)

    batch_size = 32
    episodes = 1000

    for episode in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        done = False
        time = 0

        while not done:
            # Render the environment (Optional, you can remove this line for faster training)
            env.render()

            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done or time == 499 else -100

            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            time += 1

            if done:
                print("Episode: {}/{}, Score: {}, Epsilon: {:.2}"
                      .format(episode + 1, episodes, time, agent.epsilon))
                break

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)


c:\Users\HP\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


ValueError: cannot reshape array of size 2 into shape (1,4)